# 🧠 Fractal Three-Body Quantum Dashboard

**Objective:** Consolidated interface for System Verification, Debugging, and Full Learning Cycle.
**Architecture:** Fractal Three-Body Quantum Engine + Quantum Bayesian Brain

**Sections:**
1. Preflight & Environment Checks (Deep Audit)
2. Data Pipeline Test
3. Core Component Tests
4. Quick Learn (Simulation)
5. Full Training Loop (Production Run)
6. Result Analysis & Visualization

## 1. Preflight & Environment Checks ✈️
Verify Python environment, PyTorch CUDA availability, and system paths.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import ipywidgets as widgets
from pathlib import Path
from IPython.display import display, clear_output
import torch

# Add root to path
current_dir = Path(os.getcwd())
project_root = current_dir.parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
print(f"Project Root: {project_root}")

# Import Core Components
from core.quantum_field_engine import QuantumFieldEngine
from core.three_body_state import ThreeBodyQuantumState
from core.bayesian_brain import QuantumBayesianBrain
from training.orchestrator import BayesianTrainingOrchestrator, DayResults
from training.databento_loader import DatabentoLoader
import config.settings

# 1. Check Operational Mode
print(f"\nOperational Mode: {config.settings.OPERATIONAL_MODE}")

# 2. Check CUDA
if torch.cuda.is_available():
    print(f"🟢 CUDA Available: YES")
    print(f"   Device: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.2f} GB")
else:
    print("🔴 CUDA Not Available. System running in CPU mode.")

print("\n--- Environment Ready ---")

## 2. Data Pipeline Test 📊
Validate loading of a single data file using DatabentoLoader.

In [ ]:
import glob

try:
    # Find data files
    data_dir = project_root / 'DATA' / 'RAW'
    files = sorted(glob.glob(str(data_dir / '*.dbn*')) + glob.glob(str(data_dir / '*.parquet')))
    
    if not files:
        print(f"🔴 No data files found in {data_dir}")
        test_file = None
    else:
        test_file = files[0]
        print(f"Loading {os.path.basename(test_file)}...")
        
        if test_file.endswith('.parquet'):
            df = pd.read_parquet(test_file)
        else:
            df = DatabentoLoader.load_data(test_file)
            
        print(f"🟢 Load Success! Shape: {df.shape}")
        display(df.head())
        
        # Store sample for quick tests
        sample_data = df.head(50000).copy() if len(df) > 50000 else df.copy()

except Exception as e:
    print(f"🔴 Pipeline Test Failed: {e}")

## 3. Core Component Tests ⚙️
Verify QuantumFieldEngine, ThreeBodyQuantumState, and QuantumBayesianBrain.

In [ ]:
print("--- Component Status ---")

# 1. QuantumFieldEngine
try:
    engine = QuantumFieldEngine()
    print(f"🟢 QuantumFieldEngine: Initialized (Device={engine.device})")
except Exception as e:
    print(f"🔴 QuantumFieldEngine: Failed ({e})")

# 2. QuantumBayesianBrain
try:
    brain = QuantumBayesianBrain()
    print("🟢 QuantumBayesianBrain: Initialized")
except Exception as e:
    print(f"🔴 QuantumBayesianBrain: Failed ({e})")

# 3. ThreeBodyQuantumState (Mock Creation)
try:
    # Create a dummy state manually to verify class structure
    state = ThreeBodyQuantumState.null_state()
    assert hasattr(state, 'tunnel_probability')
    print("🟢 ThreeBodyQuantumState: Verified")
except Exception as e:
    print(f"🔴 ThreeBodyQuantumState: Failed ({e})")

## 4. Quick Learn: Simulation 🎲
Run a quick simulation on the sample data using BayesianTrainingOrchestrator.

In [ ]:
class Config:
    def __init__(self):
        self.iterations = 5
        self.max_days = 1
        self.checkpoint_dir = str(project_root / 'checkpoints')
        self.no_dashboard = True # Use inline plots instead
        self.data = str(test_file) if 'test_file' in locals() and test_file else ""

def run_quick_sim(b):
    with out_area:
        clear_output()
        if 'sample_data' not in globals():
            print("🔴 Sample data not loaded. Run Section 2 first.")
            return

        print("Initializing Quick Simulation (5 iterations)...")
        cfg = Config()
        
        try:
            # Initialize Orchestrator
            orch = BayesianTrainingOrchestrator(cfg)
            
            # Run on sample data
            results = orch.train(sample_data)
            
            print("\n🟢 Simulation Complete!")
            if results:
                res = results[0]
                print(f"Best Iteration: {res.best_iteration}")
                print(f"Win Rate: {res.best_win_rate:.1%}")
                print(f"PnL: ${res.best_pnl:.2f}")
                print(f"Trades: {res.total_trades}")
            else:
                print("No results returned.")
            
        except Exception as e:
            print(f"🔴 Simulation Failed: {e}")
            import traceback
            traceback.print_exc()

btn_run = widgets.Button(description="Run Quick Sim", button_style='primary')
out_area = widgets.Output()

btn_run.on_click(run_quick_sim)
display(widgets.VBox([btn_run, out_area]))

## 5. Full Training Loop 🚀
Execute the full training loop with configurable iterations.

In [ ]:
iter_slider = widgets.IntSlider(value=50, min=10, max=1000, step=10, description='Iterations:')
btn_full = widgets.Button(description="Start Training", button_style='success')
full_out = widgets.Output()

def start_full_training(b):
    with full_out:
        clear_output()
        print(f"Starting Full Training with {iter_slider.value} iterations per day...")
        
        if 'df' not in globals():
             print("🔴 Data not loaded. Run Section 2 first.")
             return

        cfg = Config()
        cfg.iterations = iter_slider.value
        cfg.max_days = None # All days
        
        try:
            orch = BayesianTrainingOrchestrator(cfg)
            results = orch.train(df)
            print("✅ Training Complete!")
        except Exception as e:
            print(f"❌ Training Error: {e}")
            import traceback
            traceback.print_exc()

btn_full.on_click(start_full_training)
display(widgets.VBox([iter_slider, btn_full, full_out]))

## 6. Result Analysis & Visualization 📈
Analyze performance metrics from the latest run.

In [ ]:
def plot_results(results):
    if not results:
        print("No results to plot.")
        return
    
    dates = [r.date for r in results]
    pnls = [r.real_pnl for r in results]
    win_rates = [r.real_win_rate for r in results]
    
    # PnL Chart
    fig_pnl = px.line(x=dates, y=np.cumsum(pnls), title='Cumulative PnL (Real Walk-Forward)', labels={'x': 'Date', 'y': 'PnL ($)'})
    fig_pnl.show()
    
    # Win Rate Chart
    fig_wr = px.bar(x=dates, y=win_rates, title='Daily Win Rate', labels={'x': 'Date', 'y': 'Win Rate'})
    fig_wr.show()

btn_plot = widgets.Button(description="Plot Results", button_style='info')
btn_plot.on_click(lambda b: plot_results(globals().get('results', [])))
display(btn_plot)